# Usage Examples

In this notebook, we provide a few examples of how to use the `subseasonal_data` package. This assumes that you have the package and `azcopy` installed. For more details, see the `Getting Started.ipynb` notebook.

**Summary:**

1. [Example: Climatology](#Example:-Climatology)
2. [Example: Ground Truth](#Example:-Ground-Truth)
3. [Example: CFSv2](#Example:-CFSv2)
3. [Example: Combined Data](#Example:-Combined-Data)

In [ ]:
# General imports
import pandas as pd
import calendar
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import matplotlib
import matplotlib.animation as animation
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib notebook

In [ ]:
from subseasonal_data import data_loaders

In [ ]:
# Utility functions
def show_measurement_on_map(data_matrix, title, vmax):
    """Show sequential measurements on the U.S. map in an matplotlib.animation plot
    
    Parameters
    ----------
    data_matrix: array of formatted data matrices (see get_data_matrix)
    
    title: array of titles to accompany the data matrices
    
    vmax: Maximum value on colorbar. Minimum is 0.
    """
    # Set figure
    fig = plt.figure(figsize=(9, 6))
    ax = fig.add_axes([0, 0, 1, 1], projection=ccrs.PlateCarree())

    # Draw coastlines, US states
    ax.coastlines(linewidth=0.2, color='black')  # add coast lines
    ax.add_feature(cfeature.STATES)  # add US states
    ax.set_yticks(np.arange(25, 50+1, 5), crs=ccrs.PlateCarree())
    ax.set_xticks(np.arange(-125, -67+1, 8), crs=ccrs.PlateCarree())
    lats = np.linspace(26, 50, data_matrix[0].shape[0]+1)
    lons = np.linspace(-125, -68, data_matrix[0].shape[1]+1)
    color_map = 'RdBu_r'
    plot = ax.pcolormesh(lons+0.5, lats-0.5, data_matrix[0],
                        vmin=0, vmax=vmax,
                        cmap=color_map, snap=True)
    cb = plt.colorbar(plot, fraction=0.02, pad=0.04)
    def animate(i):
        plot.set_array(data_matrix[i].ravel())
        plt.title(title[i])
        return plot
    ani = animation.FuncAnimation(
        fig, animate, frames=len(data_matrix), interval=700, blit=False, repeat=False)
    return ani
    
def get_data_matrix(data, values):
    """Get pandas dataframe with (lat, lon, values) ready for plotting
    
    If there is more than one value per (lat, lon) grid point, the values will be averaged.
    This is especially useful for calculating daily/monthly/yearly averages.
    
    Parameters
    ----------
    data: pd.DataFrame with (lat, lon, values) format
    
    values: Name of the 'values' column
    """
    # Average if more than one data point per (lat, lon) pair
    data_aux = data[["lat", "lon", values]].groupby(by=["lat", "lon"], as_index=False).agg(np.mean)
    data_pivot = data_aux.pivot(index='lat', columns='lon', values=values)
    data_matrix = data_pivot.values
    data_matrix = np.ma.masked_invalid(data_matrix)
    return data_matrix

# Example: Climatology

We load climatology data for temperature, i.e. historical average of temperatures for each grid point. We plot the historical averages for the 1st of each month.

In [ ]:
# Load data
df = data_loaders.get_climatology("us_tmp2m")

In [ ]:
df.head()

In [ ]:
# Process data
df_slice = df[pd.DatetimeIndex(df.start_date).day==1]
data_matrix = [
    get_data_matrix(df_slice[pd.DatetimeIndex(df_slice.start_date).month==i], values="tmp2m") for i in range(1, 13)
]
title = [f"Average temperature (C) for {calendar.month_name[i]} 1st" for i in range(1, 13)]

In [ ]:
# Plot
show_measurement_on_map(data_matrix, title, vmax=25)

# Example: Ground Truth

We load ground truth data for precipitation for each grid point. We plot the yearly averages for each grid point.

In [ ]:
# Load data
df = data_loaders.get_ground_truth("us_precip")

In [ ]:
df.head()

In [ ]:
year_range = np.arange(1980, 2021, 5)
df_slice = df[pd.DatetimeIndex(df.start_date).year.isin(year_range)]

data_matrix = [
    get_data_matrix(df_slice[pd.DatetimeIndex(df_slice.start_date).year==i], values="precip") for i in year_range
]
title = [f"Yearly average precipitation (mm): {i}" for i in year_range]

In [ ]:
show_measurement_on_map(data_matrix, title, vmax=100)

# Example: CFSv2

CFSv2 is one of the SubX models. It has predicitions up to 42 days ahead (leads). We load the dataset with includes 2-day averages for CFSv2 temperatures.

In [ ]:
df = data_loaders.get_forecast("subx_cfsv2-tmp2m-us")

In [ ]:
df.head()

# Example: Combined Data

We load the dataset that contains most of the data made available. 

In [ ]:
df = data_loaders.load_combined_data("all_data", "us_tmp2m", "34w")

In [ ]:
df.columns